# Código para determinar el poder de frenado mediante la ecuación de Bethe-Bloch:

Importación de librerías, definición de constantes y fórmulas a utilizar:

In [ ]:
# Librerías
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Constantes:
N_A = 6.02e23
m_e = 0.511
K = 9.98e-29

In [ ]:
# Fórmulas:
def Gamma(T,M):   #Factor gamma de la partícula incidente
  return (T+M)/M
def Beta(G):    # Factor beta de la partícula incidente
  return np.sqrt(1-1/G**2)
def N(rho,A):   # Número de átomos del material
  return rho*N_A/A
def I(Z):   # Potencial de excitación media
  return (9.76+58.8*Z**(-1.19))*Z*1e-6
def BetheBloch(z,beta,gamma,N,Z,I):
  return K*(z**2)*(m_e/beta**2)*N*Z*(np.log((2*m_e*(beta**2)*(gamma**2))/I)-beta**2)

Definición de las variables:

In [ ]:
print(f'='*80)
print('DATOS DE LA PARTÍCULA INCIDENTE:')
T = float(input('- Energía cinética de la partícula [MeV]: '))
M = float(input('- Masa en reposo de la partícula [MeV/c^2]: '))
z = float(input('- Carga eléctrica de la partícula (z): '))
print(f'-'*80)
# Cálculos previos a la ecuación de BB
print(f'Factores relativistas para una partícula incidente de {T} MeV:')
G = Gamma(T,M)
B = Beta(G)
print(f'- Factor de Lorentz de la partícula: {G:.3f}')
print(f'- Velocidad relativa de la partícula: {B:.3f}')
print(f'='*80)
# --------------------------------------------------------
print('DATOS DEL MATERIAL DEL BLANCO:')
rho = float(input('- Densidad del material [kg/m^3]: '))
Z = float(input('- Número atómico del material (Z): '))
A = float(input('- Masa molar del material [g/mol]: '))
print(f'-'*80)
# Cálculos previos a la ecuación de BB
N = N(rho*1e3,A)
I = I(Z)
print(f'- Densidad de átomos del material: {N:.3e} átomos/m^2')
print(f'- Potencial de excitación media del material: {I:.3e} MeV')
print(f'='*80)

DATOS DE LA PARTÍCULA INCIDENTE:
- Energía cinética de la partícula [MeV]: 100
- Masa en reposo de la partícula [MeV/c^2]: 938.3
- Carga eléctrica de la partícula (z): 1
--------------------------------------------------------------------------------
Factores relativistas para una partícula incidente de 100.0 MeV:
- Factor de Lorentz de la partícula: 1.107
- Velocidad relativa de la partícula: 0.428
DATOS DEL MATERIAL DEL BLANCO:
- Densidad del material [kg/m^3]: 8960
- Número atómico del material (Z): 29
- Masa molar del material [g/mol]: 63
--------------------------------------------------------------------------------
- Densidad de átomos del material: 8.562e+28 átomos/m^2
- Potencial de excitación media del material: 3.141e-04 MeV


Cálculo del stopping power lineal y másico:

In [ ]:
print(f'='*100)
# Cálculo de la ecuación de Bethe-Bloch
print('STOPPING POWER LINEAL')
BB = BetheBloch(z,B,G,N,Z,I)
print(f'- Poder de frenado lineal de la partícula incidente en el material: {BB:.3e} MeV/m')
print('STOPPING POWER MÁSICO')
print(f'- Stopping power másico de la partícula incidente en el material: {BB/rho:.3e} MeV m^2/kg')
print(f'='*100)

STOPPING POWER LINEAL
- Poder de frenado lineal de la partícula incidente en el material: 4.427e+03 MeV/m
STOPPING POWER MÁSICO
- Stopping power másico de la partícula incidente en el material: 4.941e-01 MeV m^2/kg


# Cálculo de la energía perdida luego de atravesar un material de cierto grosor *t (m)*:

In [21]:
# Librerías
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Constantes:
N_A = 6.02e23
m_e = 0.511
K = 9.98e-29

# Datos y parámetros iniciales:
T_i = T
N_segmentos = 100
t = 0.0001

def StoppingPower(T):
  G = Gamma(T,M)
  B = Beta(G)
  return BetheBloch(z,B,G,N,Z,I)

def PerdidaEnergetica(T_i, t, N_segmentos):
  delta_x = t/N_segmentos
  delta_E = 0.0
  T_actual = T_i
  for i in range(N_segmentos):
    x_i = i*delta_x
    x = x_i + 0.5*delta_x
    dEdx = StoppingPower(T_actual)
    delta_E += dEdx*delta_x
    T_actual -= dEdx*delta_x
    if T_actual <= 0:
      print(f'LA PARTÍCULA SE HA DETENIDO DESPUÉS DE: {x:.3e} m')
      break
  return delta_E

Delta_E = PerdidaEnergetica(T_i, t, N_segmentos)
print(f'='*100)
print(f"Energía inicial de la partícula: {T_i} MeV")
print(f"Grosor del material: {t} m")
print(f"Número de segmentos (N): {N_segmentos}")
print(f"Energía perdida (ΔE): {Delta_E:.6f} MeV")
print(f"Energía final de la partícula: {T_i - Delta_E:.6f} MeV")
print(f'='*100)

Energía inicial de la partícula: 100.0 MeV
Grosor del material: 0.0001 m
Número de segmentos (N): 100
Energía perdida (ΔE): 0.443425 MeV
Energía final de la partícula: 99.556575 MeV


# Cálculo del grosor necesario para disminuir cierta energía de un haz incidente:

In [ ]:
# Librerías
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Constantes:
N_A = 6.02e23
m_e = 0.511
K = 9.98e-29

# Fórmulas:
def Gamma(T,M):   #Factor gamma de la partícula incidente
  return (T+M)/M
def Beta(G):    # Factor beta de la partícula incidente
  return np.sqrt(1-1/G**2)
def N(rho,A):   # Número de átomos del material
  return rho*N_A/A
def I(Z):   # Potencial de excitación media
  return (9.76+58.8*Z**(-1.19))*Z*1e-6
def BetheBloch(z,beta,gamma,N,Z,I):
  return K*(z**2)*(m_e/beta**2)*N*Z*(np.log((2*m_e*(beta**2)*(gamma**2))/I)-beta**2)
def StoppingPower(T):
  G = Gamma(T,M)
  B = Beta(G)
  return BetheBloch(z,B,G,N,Z,I)

print('DATOS DE LA PARTÍCULA INCIDENTE:')
M = float(input('- Masa en reposo de la partícula [MeV/c^2]: '))
z = float(input('- Carga eléctrica de la partícula (z): '))
print(f'-'*80)
# --------------------------------------------------------
print('DATOS DEL MATERIAL DEL BLANCO:')
rho = float(input('- Densidad del material [kg/m^3]: '))
Z = float(input('- Número atómico del material (Z): '))
A = float(input('- Masa molar del material [g/mol]: '))
print(f'-'*80)
# Cálculos previos a la ecuación de BB
N = N(rho*1e3,A)
I = I(Z)
print(f'- Densidad de átomos del material: {N:.3e} átomos/m^2')
print(f'- Potencial de excitación media del material: {I:.3e} MeV')
print(f'='*80)

# Datos y parámetros iniciales:
print('PARÁMETROS INICIALES:')
T_i = float(input('Energía cinética inicial del haz: '))
T_f = float(input('Energía cinética final del haz: '))
N_segmentos = 10
t = 0.0001

def CalculoEspesor(T_i, T_f, N_segmentos):
  delta_E = np.abs(T_f-T_i)/N_segmentos
  delta_x = 0.0
  T_actual = T_i
  for i in range(N_segmentos):
    E_i = T_actual + i*delta_E
    E = E_i + 0.5*delta_E
    dEdx = StoppingPower(E)
    delta_x += rho*(1/(dEdx))*delta_E
    T_actual -= dEdx*delta_x
  return delta_x

Delta_x = CalculoEspesor(T_i, T_f, N_segmentos)
print(f'='*100)
print(f"Energía inicial de la partícula: {T_i} MeV")
print(f"Energía final de la partícula: {T_f} MeV")
print(f"Grosor del material para frenar ΔE: {Delta_x} m")
print(f"Número de segmentos (N): {N_segmentos}")
print(f'='*100)

DATOS DE LA PARTÍCULA INCIDENTE:
- Masa en reposo de la partícula [MeV/c^2]: 938.3
- Carga eléctrica de la partícula (z): 1
--------------------------------------------------------------------------------
DATOS DEL MATERIAL DEL BLANCO:
- Densidad del material [kg/m^3]: 8960
- Número atómico del material (Z): 29
- Masa molar del material [g/mol]: 63
--------------------------------------------------------------------------------
- Densidad de átomos del material: 8.562e+28 átomos/m^2
- Potencial de excitación media del material: 3.141e-04 MeV
PARÁMETROS INICIALES:
Energía cinética inicial del haz: 600
Energía cinética final del haz: 400
Energía inicial de la partícula: 600.0 MeV
Energía final de la partícula: 400.0 MeV
Grosor del material para frenar ΔE: 679.5590037780458 m
Número de segmentos (N): 10
